<a href="https://colab.research.google.com/github/emmanuelinfante/SubtitlesEveryone/blob/main/SubtitlesEveryone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**📝¡SubtitlesForEveryone!** Transcribe and translate for free

Welcome to the official **SubtitlesForEveryone** notebook!

>This notebook compiles and facilitates the use of artificial intelligence tools for audio transcription and translation. If you need to create subtitles for your movie or video, you just have to follow the instructions I'll provide below, and you'll see how easy it is.

## How to Get Started? 🚀

1. **Enable the GPU**: For fast transcription, make sure to enable the GPU. Go to "Runtime" > "Change runtime type" > and select "T4 GPU" in the "Hardware accelerator" option.
2. **Upload your audio or Paste a YouTube URL**: Use the tool below to upload your audio file or use the tool to import your YouTube video.
3. **Run the cells**: Simply run the cells in order, and you'll see the magic happen! ✨

**Note**: If you're new to Google Colab, each code cell is executed by clicking the play button (▶️) on the left of the cell, or you can press `Shift + Enter`.

# **1. Installation of necessary packages! 📦**
First, we need to download all the necessary libraries to make this notebook work!

In [ ]:
#Installation of lightweight but necessary packages"
!pip install srtranslator
!pip3 install ip-rotator
!pip install yt-dlp
!pip install srt
!pip install translators

In [ ]:
# Installation of fundamental packages
!pip install demucs
!pip install git+https://github.com/m-bain/whisperx

# **Now, create your subtitle!** 📜

In [ ]:
#@markdown #**1. Upload your audio file 📻**

#@markdown Click the button below to upload your audio file. Make sure it is an MP3 format file.

from google.colab import files

uploaded = files.upload()
audio_file = list(uploaded.keys())[0]

# Renombra el archivo de audio
import os
os.rename(audio_file, "audio.mp3")

In [ ]:
import import sys
import warnings
import yt_dlp
import subprocess
import torch
import shutil
import numpy as np
from IPython.display import display, Markdown, YouTubeVideo

device = torch.device('cuda:0')
print('Using device:', device, file=sys.stderr)

Type = "Youtube video or playlist"
#@markdown # Download the audio from your YouTube video 📀
URL = "" #@param {type:"string"}

video_path_local_list = []

if Type == "Youtube video or playlist":

  ydl_opts = {
    'format': 'm4a/bestaudio/best',
    'outtmpl': 'audio',  # Rename the extracted MP3 file to "audio.mp3"
    'postprocessors': [{
      'key': 'FFmpegExtractAudio',
      'preferredcodec': 'mp3',
    }]
  }

  with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    error_code = ydl.download([URL])
    list_video_info = [ydl.extract_info(URL, download=False)]

#change sample rate
import librosa
import soundfile as sf

def convertir_frecuencia(archivo_entrada, archivo_salida):
  # Cargar el archivo de audio
  senal, frecuencia_muestreo = librosa.load(archivo_entrada)

  # Convertir a mono
  senal_mono = librosa.to_mono(senal)

  # Nueva frecuencia de muestreo
  nueva_frecuencia = 16000

  # Convertir la señal a la nueva frecuencia
  senal_convertida = librosa.resample(senal, orig_sr=frecuencia_muestreo, target_sr=nueva_frecuencia)

  # Guardar el archivo de audio convertido
  sf.write(archivo_salida, senal_convertida, nueva_frecuencia)

# Ejemplo de uso
archivo_entrada = "audio.mp3"
archivo_salida = "audio_16_000.mp3"

convertir_frecuencia(archivo_entrada, archivo_salida)

In [ ]:
#@markdown # *Extract vocals From Audio (Alpha)*
#@markdown (Optional, but <font color=4bb340> **Highly Recommended** <font color=d5d5d5> to use):
#@markdown - Great Improvement In Eastern Languages!
#@markdown - Prevents WhisperX from Confusing Background Sounds with Speech.
#@markdown - Enhances transcription with any language.
#@markdown - Thanks to this extraction, it will be very rare and unlikely that you have errors in your transcription.

#@markdown **Only issue:** It still cannot be used with one-hour long audios. It works very well with 30-minute audios.
!python -m demucs.separate /content/audio_16_000.mp3 --mp3 --mp3-bitrate 128 -n hdemucs_mmi --out /content --mp3-preset 7 -d cuda --two-stems=vocals

In [ ]:
#@markdown # **2. Voice Transcription ✍️**
#@markdown Now, let's transcribe your audio file.
!whisperx /content/audio.mp3 --model large-v3 --output_dir . --device cuda --fp16 True --batch_size 16 --compute_type float16 --beam_size 10 --print_progress True --chunk_size 10 --no_align

### Descargar el resultado 📂
Download_file = False # @param{type:"boolean"}
if Download_file:
 from google.colab import files
 files.download ("audio.srt")

In [ ]:
#@markdown # **3. Translate your Subtitle! 🌍**
#@markdown Transcribe your subtitle and translate it into another language. You can use more than 6 translators to choose from.

import ip_rotator
proxy = ip_rotator.Proxy(https = True)
import translators
import srt

# Read the original SRT file
with open("audio.srt", "r", encoding="utf-8") as f:
    subtitles = srt.parse(f.read())

# Function to translate a subtitle (consider error handling)
def translate_subtitle(subtitle):
    try:
        translated_content = translators.translate_text(
            query_text=subtitle.content,
            from_language="auto"#@param {type:"string"}
            ,
            to_language="en"#@param {type:"string"}
            ,
            translator="deepl" #@param ['alibaba', 'apertium', 'argos', 'baidu', 'bing', 'caiyun', 'cloudTranslation', 'elia', 'google', 'hujiang', 'iciba', 'iflytek', 'iflyrec', 'itranslate', 'judic', 'languageWire', 'lingvanex', 'niutrans', 'mglip', 'mirai', 'modernMt', 'myMemory', 'papago', 'qqFanyi', 'qqTranSmart', 'reverso', 'sogou', 'sysTran', 'tilde', 'translateCom', 'translateMe', 'utibet', 'volcEngine', 'yandex', 'yeekit', 'youdao']{allow-input: true}
        )
        print(f" Subtitile Results {subtitle.index}:")
        print(f"  Original: {subtitle.content}")
        print(f"  Traduccion: {translated_content}")
        return srt.Subtitle(subtitle.index, subtitle.start, subtitle.end, translated_content)
    except Exception as e:  # Catch potential translation errors
        print(f"Error traduciendo subtítulo {subtitle.index}: {e}")
        return subtitle  # Return the original subtitle if translation fails

# Translate each subtitle
translated_subtitles = [translate_subtitle(subtitle) for subtitle in subtitles]

# Write the translated subtitles to a new file
srt_content = srt.compose(translated_subtitles)
with open("translated.srt", "w", encoding="utf-8") as f:
    f.write(srt_content)

### Download result 📂
download_file = False # @param{type:"boolean"}
if download_file:
 from google.colab import files
 files.download ("translated.srt")

In [ ]:
#@markdown # <font color=somblue>**DeepL Unlimited Subtitles!** <font color=d5d5d5> *Best Translator.*
#@markdown Translate your subtitles with great precision using DeepL Unlimitedly!
Source_Language = "auto" # @param ['ar', 'auto', 'bg', 'cs', 'da', 'de', 'el', 'en', 'es', 'et', 'fi', 'fr', 'hu', 'id', 'it', 'ja', 'ko', 'lt', 'lv', 'nb', 'nl', 'pl', 'pt', 'ro', 'ru', 'sk', 'sl', 'sv', 'tr', 'uk', 'zh']{allow-input: true}

#@title Target Language
Target_Language = "es" #@param ['ar', 'bg', 'cs', 'da', 'de', 'el', 'en', 'es', 'et', 'fi', 'fr', 'hu', 'id', 'it', 'ja', 'ko', 'lt', 'lv', 'nb', 'nl', 'pl', 'pt', 'ro', 'ru', 'sk', 'sl', 'sv', 'tr', 'uk', 'zh']{allow-input: true}

import os

# SRT File
from srtranslator import SrtFile

from srtranslator.translators.pydeeplx import PyDeepLX
translator = PyDeepLX()
filepath = "/content/audio.srt"

# SRT File
sub = SrtFile(filepath)
# Translate
sub.translate(translator, Source_Language, Target_Language)

# Making the result subtitles prettier
sub.wrap_lines()

sub.save(f"{os.path.splitext(filepath)[0]}_translated.srt")

### Download result 📂
Download_file = False # @param{type:"boolean"}
if Download_file:
from google.colab import files
files.download ("audio_translated.srt")
#@markdown **Warning:**
#@markdown *Using this translator will ensure very good translations, but it's not fast. The duration of the translation process depends on the length of your subtitle to translate.*

## **Languages available for DeepL. 🗨️**

```
'ar': Arabic
'auto': Auto
'bg': Bulgaria
'cs': Czech
'da': Danish
'de': German
'el': Greek
'en': English
'es': Spanish
'et': Estonian
'fi': Finnish
'fr': French
'hu': Hungarian
'id': Indonesian
'it': Italian
'ja': Japanese
'ko': Korean
'lt': Lithuanian
'lv': Latvian
'nb': Norwegian Bokmål
'nl': Dutch
'pl': Polish
'pt': Portuguese
'ro': Romanian
'ru': Russian
'sk': Slovak
'sl': Slovenian
'sv': Swedish
'tr': Turkish
'uk': Ukrainian
'zh': Chinese
```



## **Credits and Special Thanks**
This notebook combines many free libraries from Github and PyPI to achieve its purpose. Below, I will share what they are:

- [**WhisperX** - Artificial Intelligence for Transcription](https://github.com/m-bain/whisperX)
- [**Translators** - Library with API for over 10 languages](https://pypi.org/project/translators/)
- [**Srt** - Necessary for reading .srt files](https://pypi.org/project/srt/)
- [**Demucs** - For voice extraction](https://pypi.org/project/demucs/)
- [**Ctranslate2** - Necessary for WhisperX](https://pypi.org/project/ctranslate2/)
- [**Yt-dlp** - for video downloading](https://pypi.org/project/yt-dlp/)
- [**Ip-Rotator** - Proxy for APIs](https://pypi.org/project/ip-rotator/)
- [**Srtranslator** - Subtitle Translation with Deepl](https://pypi.org/project/srtranslator/)

Important Clarification, this notebook also uses its own code for subtitle creation; Translators **was not created to translate subtitles, I had to write the code myself to make it work that way.** Still, I greatly appreciate the team behind all these projects, they are crucial to making this project possible! With nothing more to say, Enjoy this project!